# This is an investigatory analysis into the sales data of a restaurant... 

It has been done mainly SQL but on top of pyspark...

So lets get started...

First we import the pyspark and the findspark modules... and also initialize the findspark module...

In [2]:
import findspark
findspark.init('/Users/nishantuzir/spark-2.3.0-bin-hadoop2.7')
import pyspark

now we initialize the SparkContext objects...

In [3]:
sc = pyspark.SparkContext()

here we read both the json files...using the wholeTextFiles method...

one thing to know here is that the wholeTextFiles method produces a tuple RDD whose 1st element is a filename and the 2nd element is the data with lines separated by whitespace. 

We use map to create the new RDD using the 2nd element of the tuple.

In [4]:
data1 = sc.wholeTextFiles('/data/orders.json').map(lambda x: x[1])
data2 = sc.wholeTextFiles('/data/order_items.json').map(lambda x: x[1])

as said earlier that the data is in the form of lines separated by whitespace, we need to remove these useless white spaces. 

That we will do using the re package...

In [5]:
import re
data1 = data1.map(lambda x: re.sub('\s+','',x))
data2 = data2.map(lambda x: re.sub('\s+','',x))

after that we import the SQLContext and initialize it...

In [6]:
from pyspark.sql import SQLContext
sqlcontext = SQLContext(sc)

now its time to conume the RDD using the SQLContext object named sqlcontext...

after that, we create a temporary table using registerTempTable and pass the name of the tables inside it..

In [7]:
orders = sqlcontext.read.json(data1)
order_items = sqlcontext.read.json(data2)
orders.registerTempTable('orders')
order_items.registerTempTable('order_items')

So we are done with the preparation part...now lets do some analysis using good old SQL running on top of Spark!!

In [8]:
sqlcontext.sql('select * from orders' ).show(5)

+------------+-----------------+------------+---+----------+---------------+
|delivered_at|delivered_at_time|delivered_to| id|ordered_at|ordered_at_time|
+------------+-----------------+------------+---+----------+---------------+
|  2015-08-19|         22:19:41|         767|  0|2015-08-19|       22:01:25|
|  2015-11-16|         00:40:18|         847|  1|2015-11-16|       00:12:29|
|  2015-11-27|         08:08:15|         972|  2|2015-11-27|       08:06:06|
|  2015-10-14|         01:25:10|         390|  3|2015-10-14|       01:16:42|
|  2015-11-28|         05:16:24|         914|  4|2015-11-28|       05:10:27|
+------------+-----------------+------------+---+----------+---------------+
only showing top 5 rows



In [9]:
sqlcontext.sql('select * from order_items').show(5)

+-----------+---+--------------------+--------+
|amount_paid| id|                name|order_id|
+-----------+---+--------------------+--------+
|        205|  0|      chicken-burger|     114|
|        225|  1|chicken-tikka-san...|    2825|
|        185|  2|almond-choco-dip-...|    4717|
|        105|  3|               juice|    1035|
|        185|  4|grilled-cheese-sa...|    1023|
+-----------+---+--------------------+--------+
only showing top 5 rows



So, everything is working just fine!! We are good to go...

Let's see how many orders are placed per day...

In [10]:
sqlcontext.sql('select ordered_at,count(1) as total_orders from orders group by 1 order by 1').show(10)

+----------+------------+
|ordered_at|total_orders|
+----------+------------+
|2015-08-09|           1|
|2015-08-11|           5|
|2015-08-12|           1|
|2015-08-13|           2|
|2015-08-14|           9|
|2015-08-15|           4|
|2015-08-16|           5|
|2015-08-17|           6|
|2015-08-18|           4|
|2015-08-19|          10|
+----------+------------+
only showing top 10 rows



Now let's see the total revenue collected from all the orders per day...

to do that, we will have to use the join command...

In [11]:
sqlcontext.sql('select ordered_at, round(sum(amount_paid),2) as revenue_collected from orders join order_items on orders.id = order_items.order_id where name = "kale-smoothie" group by ordered_at order by ordered_at').show(20)


+----------+-----------------+
|ordered_at|revenue_collected|
+----------+-----------------+
|2015-08-23|              175|
|2015-08-26|              175|
|2015-08-27|              175|
|2015-09-01|              175|
|2015-09-03|              175|
|2015-09-05|              175|
|2015-09-10|              175|
|2015-09-11|              175|
|2015-09-12|              175|
|2015-09-13|              175|
|2015-09-20|              350|
|2015-09-23|              175|
|2015-09-26|              525|
|2015-09-28|              175|
|2015-09-29|              175|
|2015-09-30|              175|
|2015-10-04|              175|
|2015-10-06|              175|
|2015-10-09|              175|
|2015-10-10|              350|
+----------+-----------------+
only showing top 20 rows



well we cant say much from this....lets break this down...

we will see the total revenue collected per food item for the entire duration of the time that has been depicted in the dataset...and we will arrannge it in descending order...

In [12]:
sqlcontext.sql('select name, round(sum(amount_paid), 2) as total_revenue from order_items group by name order by 2 desc').show()

+--------------------+-------------+
|                name|total_revenue|
+--------------------+-------------+
|chicken-tikka-san...|      1130400|
|grilled-cheese-sa...|       770155|
|      chicken-burger|       711350|
|almond-choco-dip-...|       525955|
|                soda|       195525|
|               juice|       104685|
|                cake|        96660|
|     banana-smoothie|        18900|
|       kale-smoothie|        12600|
+--------------------+-------------+



it seems to be somewhat comprehensible now... 

lets now see the percentage of revenue each of the food items represent...

this will give a better idea..

In [13]:
sqlcontext.sql('select name, round(sum(amount_paid) /(select sum(amount_paid) from order_items) * 100.0, 2) as pct from order_items group by 1 order by 2 desc').show()



+--------------------+-----+
|                name|  pct|
+--------------------+-----+
|chicken-tikka-san...| 31.7|
|grilled-cheese-sa...| 21.6|
|      chicken-burger|19.95|
|almond-choco-dip-...|14.75|
|                soda| 5.48|
|               juice| 2.94|
|                cake| 2.71|
|     banana-smoothie| 0.53|
|       kale-smoothie| 0.35|
+--------------------+-----+



whoa!! looks like smoothies are not bringing in much revenue!

lets be absolutely sure about this...

to do this we need to group the food items into food categries such as these sandwich, burger, juice etc...

In [14]:
sqlcontext.sql('select *,case name when "kale-smoothie" then "smoothie" when "banana-smoothie" then "smoothie" when "orange-juice" then "drink" when "soda" then "drink" when "almond-choco-dip-biscotti" then "desert" when "grilled-cheese-sandwich" then "sandwich" when "chicken-tikka-sandwich" then "sandwich" when "chicken-burger" then "burger" else "desert" end as category from order_items order by id limit 100').show()


+-----------+---+--------------------+--------+--------+
|amount_paid| id|                name|order_id|category|
+-----------+---+--------------------+--------+--------+
|        205|  0|      chicken-burger|     114|  burger|
|        225|  1|chicken-tikka-san...|    2825|sandwich|
|        185|  2|almond-choco-dip-...|    4717|  desert|
|        105|  3|               juice|    1035|  desert|
|        185|  4|grilled-cheese-sa...|    1023|sandwich|
|        205|  5|      chicken-burger|    4359|  burger|
|        225|  6|chicken-tikka-san...|    3929|sandwich|
|        205|  7|      chicken-burger|    3704|  burger|
|        185|  8|grilled-cheese-sa...|    1666|sandwich|
|        225|  9|chicken-tikka-san...|    1477|sandwich|
|        225| 10|chicken-tikka-san...|    4369|sandwich|
|        185| 11|grilled-cheese-sa...|     998|sandwich|
|        205| 12|      chicken-burger|    2730|  burger|
|        225| 13|chicken-tikka-san...|    3038|sandwich|
|        205| 14|      chicken-

now we will see the percentage sales of each of the categories of food items that we prepared in the last command...

In [15]:
sqlcontext.sql('select case name when "kale-smoothie" then "smoothie" when "banana-smoothie" then "smoothie" when "orange-juice" then "drink" when "soda" then "drink" when "almond-choco-dip-biscotti" then "desert" when "grilled-cheese-sandwich" then "sandwich" when "chicken-tikka-sandwich" then "sandwich" when "chicken-burger" then "burger" else "desert" end as category, round(1.0 * sum(amount_paid) /(select sum(amount_paid) from order_items) * 100, 2) as pct from order_items group by 1 order by 2 desc').show(20)


+--------+-----+
|category|  pct|
+--------+-----+
|sandwich|53.29|
|  desert|20.39|
|  burger|19.95|
|   drink| 5.48|
|smoothie| 0.88|
+--------+-----+



So it looks like smoothies are really not bringing in the big bucks for the restaurant...

So should they remove the items al together???

Well, lets get a closer look..

Infact, before taking them out of the menu, we need to figure out how many customers ordered them...

In [16]:
sqlcontext.sql('select name, count(distinct order_id) as distinct_order_ids from order_items group by 1 order by 2 desc').show()

+--------------------+------------------+
|                name|distinct_order_ids|
+--------------------+------------------+
|chicken-tikka-san...|              3168|
|grilled-cheese-sa...|              2832|
|      chicken-burger|              2487|
|almond-choco-dip-...|              2175|
|                soda|              2041|
|               juice|               905|
|                cake|               669|
|     banana-smoothie|               105|
|       kale-smoothie|                72|
+--------------------+------------------+



well looks like smoothies are not ordered by many people, especially kale-smoothies!

What might be the reason? Don't they like it?

what about the 72 people who ordered the smoothie in the course of 5 months...!

Lets have a look at the reorder rate of kale-smoothie...

the reorder rate can be defined as the ratio of total number of distinct orders for a food item to the total number customers purchasing them...if the ratio is high that means a high reorder rate and vice-versa...

In [17]:
sqlcontext.sql('select name, round(1.0 * count(distinct order_id) / count(delivered_to), 2) as reorder_rate from order_items join orders on orders.id = order_items.order_id group by 1 order by 2 desc').show()


+--------------------+------------+
|                name|reorder_rate|
+--------------------+------------+
|       kale-smoothie|        1.00|
|     banana-smoothie|        0.97|
|                cake|        0.93|
|               juice|        0.91|
|                soda|        0.78|
|almond-choco-dip-...|        0.76|
|      chicken-burger|        0.72|
|grilled-cheese-sa...|        0.68|
|chicken-tikka-san...|        0.63|
+--------------------+------------+



well well well!!!

we can clearly see that the smoothies have a very high reorder rate!!

so definitely cannot be taken out of the menu as doing so would upset a lot of loyal smoothie loving customers!

so how to increase the sales?     to be continued...